In [23]:
import re
# from rich import print

In [2]:
text = "Paul Newman was an American Actor, but Paul Hollywood is a British TV Host. The name Paul is quite common."

pattern = r"Paul [A-Z]\w+"

matches = re.finditer(pattern, text)

for match in matches:
    print(match)

<re.Match object; span=(0, 11), match='Paul Newman'>
<re.Match object; span=(39, 53), match='Paul Hollywood'>


In [3]:
import spacy
from spacy.tokens import Span

In [24]:
nlp = spacy.blank("en")
doc = nlp(text)

original_ents = list(doc.ents)

mwt_ents = []

for match in re.finditer(pattern, doc.text):
    start, end = match.span()
    span = doc.char_span(start,end)
    if span is not None:
        mwt_ents.append((span.start, span.end, span.text))

for ent in mwt_ents:
    start, end, name = ent
    per_ent = Span(doc, start, end, label="PERSON")
    original_ents.append(per_ent)

doc.ents = original_ents

print(doc.ents)

for ent in doc.ents:
    print(ent.text, ent.label_)


(Paul Newman, Paul Hollywood)

Paul Newman PERSON

Paul Hollywood PERSON

In [13]:
print(mwt_ents)

[(0, 2, 'Paul Newman'), (8, 10, 'Paul Hollywood')]

In [6]:
from spacy.language import Language

@Language.component("paul_ner")

def paul_ner(doc):
    original_ents = list(doc.ents)
    pattern = r"Paul [A-Z]\w+"
    mwt_ents = []

    for match in re.finditer(pattern, doc.text):
        start, end = match.span()
        span = doc.char_span(start,end)
        if span is not None:
            mwt_ents.append((span.start, span.end, span.text))

    for ent in mwt_ents:
        start, end, name = ent
        per_ent = Span(doc, start, end, label="PERSON")
        original_ents.append(per_ent)

    doc.ents = original_ents
    
    return doc
    

In [ ]:
nlp.remove_pipe("paul_ner")

In [16]:
nlp.add_pipe("paul_ner")

<function __main__.paul_ner(doc)>

In [18]:
print(nlp.analyze_pipes())

{
    'summary': {
        'paul_ner': {'assigns': [], 'requires': [], 'scores': [], 'retokenizes': False}
    },
    'problems': {'paul_ner': []},
    'attrs': {}
}

In [19]:
nlp.analyze_pipes()

{'summary': {'paul_ner': {'assigns': [],
   'requires': [],
   'scores': [],
   'retokenizes': False}},
 'problems': {'paul_ner': []},
 'attrs': {}}

In [ ]:
text = "Paul Newman was an American Actor, but Paul Hollywood is a British TV Host. The name Paul is quite common."
doc = nlp(text)
for ent in doc.ents:
    print(ent.text, ent.label_)

Paul Newman PERSON
Paul Waswas PERSON
Paul Hollywood PERSON


In [30]:
from spacy.language import Language
from spacy.util import filter_spans

@Language.component("cinema_ner")

def cinema_ner(doc):
    pattern = r"Hollywood"
    original_ents = list(doc.ents)
    mwt_ents = []

    for match in re.finditer(pattern,doc.text):
        start,end = match.span()
        span = doc.char_span(start,end)
        if span is not None:
            mwt_ents.append((span.start, span.end, span.text))
    
    for ent in mwt_ents:
        start, end, name = ent
        per_ent = Span(doc, start, end, label="CINEMA")
        original_ents.append(per_ent)
    
    filtered = filter_spans(original_ents)

    doc.ents = filtered

    return doc


In [31]:
nlp2 = spacy.load("en_core_web_sm")
nlp2.add_pipe("cinema_ner")

<function __main__.cinema_ner(doc)>

In [32]:
doc2 = nlp2(text)
for ent in doc2.ents:
    print(ent.text, ent.label_)

Paul Newman PERSON
American NORP
Paul Hollywood PERSON
British NORP
Paul PERSON


In [20]:
nlpsm = spacy.load("en_core_web_sm")

In [21]:
nlpsm.analyze_pipes()

{'summary': {'tok2vec': {'assigns': ['doc.tensor'],
   'requires': [],
   'scores': [],
   'retokenizes': False},
  'tagger': {'assigns': ['token.tag'],
   'requires': [],
   'scores': ['tag_acc'],
   'retokenizes': False},
  'parser': {'assigns': ['token.dep',
    'token.head',
    'token.is_sent_start',
    'doc.sents'],
   'requires': [],
   'scores': ['dep_uas',
    'dep_las',
    'dep_las_per_type',
    'sents_p',
    'sents_r',
    'sents_f'],
   'retokenizes': False},
  'attribute_ruler': {'assigns': [],
   'requires': [],
   'scores': [],
   'retokenizes': False},
  'lemmatizer': {'assigns': ['token.lemma'],
   'requires': [],
   'scores': ['lemma_acc'],
   'retokenizes': False},
  'ner': {'assigns': ['doc.ents', 'token.ent_iob', 'token.ent_type'],
   'requires': [],
   'scores': ['ents_f', 'ents_p', 'ents_r', 'ents_per_type'],
   'retokenizes': False}},
 'problems': {'tok2vec': [],
  'tagger': [],
  'parser': [],
  'attribute_ruler': [],
  'lemmatizer': [],
  'ner': []},
 'att

In [22]:
print(nlpsm.analyze_pipes())

{
    'summary': {
        'tok2vec': {
            'assigns': ['doc.tensor'],
            'requires': [],
            'scores': [],
            'retokenizes': False
        },
        'tagger': {
            'assigns': ['token.tag'],
            'requires': [],
            'scores': ['tag_acc'],
            'retokenizes': False
        },
        'parser': {
            'assigns': ['token.dep', 'token.head', 'token.is_sent_start', 'doc.sents'],
            'requires': [],
            'scores': [
                'dep_uas',
                'dep_las',
                'dep_las_per_type',
                'sents_p',
                'sents_r',
                'sents_f'
            ],
            'retokenizes': False
        },
        'attribute_ruler': {
            'assigns': [],
            'requires': [],
            'scores': [],
            'retokenizes': False
        },
        'lemmatizer': {
            'assigns': ['token.lemma'],
            'requires': [],
            'scores': ['lemma_acc'],
            'retokenizes': False
        },
        'ner': {
            'assigns': ['doc.ents', 'token.ent_iob', 'token.ent_type'],
            'requires': [],
            'scores': ['ents_f', 'ents_p', 'ents_r', 'ents_per_type'],
            'retokenizes': False
        }
    },
    'problems': {
        'tok2vec': [],
        'tagger': [],
        'parser': [],
        'attribute_ruler': [],
        'lemmatizer': [],
        'ner': []
    },
    'attrs': {
        'doc.tensor': {'assigns': ['tok2vec'], 'requires': []},
        'token.is_sent_start': {'assigns': ['parser'], 'requires': []},
        'token.ent_type': {'assigns': ['ner'], 'requires': []},
        'token.tag': {'assigns': ['tagger'], 'requires': []},
        'token.ent_iob': {'assigns': ['ner'], 'requires': []},
        'token.dep': {'assigns': ['parser'], 'requires': []},
        'doc.sents': {'assigns': ['parser'], 'requires': []},
        'doc.ents': {'assigns': ['ner'], 'requires': []},
        'token.head': {'assigns': ['parser'], 'requires': []},
        'token.lemma': {'assigns': ['lemmatizer'], 'requires': []}
    }
}